In [2]:
# modules
import os
import pandas as pd
import numpy as np
from tqdm import  tqdm
from os.path import basename as bn, join, split as sp

from sklearn.utils import shuffle

# DATASET Path and Constants

In [3]:
# Read paths
ROOT_TIMIT_DATA_PATH = "/home/jeevan/datasets/TIMIT Acoustic-Phonetic Continuous Speech Corpus (LDC93S1)/TIMIT"

# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Vowel info Export CSV filename
ALL_TIMIT_VOWELS_IMP_FILENAME = "a_all-timit_vowels.csv"
ALL_TIMIT_VOWELS_IMP_FILEPATH = join(ALL_EXP_FOLDER, ALL_TIMIT_VOWELS_IMP_FILENAME)

# Vowel subset Export CSV filename
SUBSET_TIMIT_VOWELS_EXP_FILENAME = "b_subset-timit_vowels_vowlimLIM.csv"
SUBSET_TIMIT_VOWELS_EXP_FILEPATH = join(ALL_EXP_FOLDER, SUBSET_TIMIT_VOWELS_EXP_FILENAME)

# TIMIT SAMPLING RATE
TIMIT_AUDIO_FS = 16000

### Import All TIMIT Vowel Info dataframe

In [4]:
ALL_TIMIT_VOWELS_DF = pd.read_csv(ALL_TIMIT_VOWELS_IMP_FILEPATH)
ALL_TIMIT_VOWEL_LIST = pd.unique(ALL_TIMIT_VOWELS_DF["vowel_name"])
ALL_TIMIT_VOWELS_DF["vowel_name"].value_counts()

ix      11587
iy       9663
ih       6760
ae       5404
eh       5293
ax       4956
axr      4790
aa       4197
ao       4096
ay       3242
ah       3185
ey       3088
ow       2913
er       2846
ux       2488
oy        947
aw        945
uh        756
uw        725
ax-h      493
Name: vowel_name, dtype: int64

## FUNCTION: Make Vowel info Subset

In [5]:
def make_vowel_subset(vowel: str, df: pd.DataFrame=ALL_TIMIT_VOWELS_DF,  limit=2) -> pd.DataFrame:
    filt = df["vowel_name"] == vowel
    vowel_df = df[filt]

    filt_sex = lambda s: vowel_df["sex"] == s
    male_vowel_df = shuffle(vowel_df[filt_sex("M")], random_state=2)
    female_vowel_df = shuffle(vowel_df[filt_sex("F")], random_state=4)

    m_size = limit if len(male_vowel_df) > limit else len(male_vowel_df)
    f_size = limit if len(female_vowel_df) > limit else len(female_vowel_df)

    male_vowel_df   = male_vowel_df[:m_size]
    female_vowel_df = female_vowel_df[:f_size]

    vowel_df = pd.concat([male_vowel_df, female_vowel_df]) 

    return vowel_df
    
make_vowel_subset(
    vowel=ALL_TIMIT_VOWEL_LIST[np.random.randint(0, len(ALL_TIMIT_VOWEL_LIST))],
    df=ALL_TIMIT_VOWELS_DF,
    limit=5
)

,index,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
75096,2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI2230.WAV,MTPR0,M,7110,9315,2205,0.444375,0.582187,0.137813,ay
24469,10,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MRCG0,M,47946,49560,1614,2.996625,3.097500,0.100875,ay
72539,0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI588.WAV,MKDB0,M,2973,4760,1787,0.185812,0.297500,0.111687,ay
59887,0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI2224.WAV,MRLD0,M,3680,7920,4240,0.230000,0.495000,0.265000,ay
14905,10,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MRRK0,M,33712,35560,1848,2.107000,2.222500,0.115500,ay
27064,6,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1873.WAV,FLMA0,F,16668,19202,2534,1.041750,1.200125,0.158375,ay
75881,3,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX438.WAV,FCEG0,F,16840,18440,1600,1.052500,1.152500,0.100000,ay
45096,11,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX212.WAV,FEEH0,F,41775,43756,1981,2.610937,2.734750,0.123813,ay
44891,10,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,FDKN0,F,38253,39720,1467,2.390813,2.482500,0.091688,ay
62976,4,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX79.WAV,FSBK0,F,19946,22391,2445,1.246625,1.399437,0.152812,ay


### Create and export Vowel Info SUBSET dataframe

In [6]:
VOWEL_LIMIT = 10
SUBSET_TIMIT_VOWELS_DF = pd.concat([make_vowel_subset(vowel, limit=VOWEL_LIMIT) for vowel in tqdm(ALL_TIMIT_VOWEL_LIST)])

SUBSET_TIMIT_VOWELS_DF.reset_index(inplace=True)
SUBSET_TIMIT_VOWELS_DF.index.name = "index"
SUBSET_TIMIT_VOWELS_DF.drop(columns=["index", "level_0"], inplace=True)

SUBSET_TIMIT_VOWELS_DF.to_csv(SUBSET_TIMIT_VOWELS_EXP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"), index=True)

SUBSET_TIMIT_VOWELS_DF 

100%|██████████| 20/20 [00:00<00:00, 87.10it/s]


,audio_filepath,wav_file,person_id,sex,start_sample,end_sample,duration_sample,start_second,end_second,duration_second,vowel_name
index,,,,,,,,,,,
0,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MTRT0,M,19040,20720,1680,1.190000,1.295000,0.105000,iy
1,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA1.WAV,MMGG0,M,31768,33269,1501,1.985500,2.079312,0.093812,iy
2,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX62.WAV,MPPC0,M,27195,28365,1170,1.699688,1.772812,0.073125,iy
3,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SA2.WAV,MKAH0,M,9800,10778,978,0.612500,0.673625,0.061125,iy
4,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI1421.WAV,MRML0,M,79791,82630,2839,4.986937,5.164375,0.177437,iy
...,...,...,...,...,...,...,...,...,...,...,...
395,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SI648.WAV,FCJF0,F,1507,2154,647,0.094187,0.134625,0.040438,ax-h
396,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX206.WAV,FVMH0,F,29660,30400,740,1.853750,1.900000,0.046250,ax-h
397,/home/jeevan/datasets/TIMIT Acoustic-Phonetic ...,SX271.WAV,FSMA0,F,15838,16318,480,0.989875,1.019875,0.030000,ax-h
